In [1]:
import pylast
import pandas as pd
import pickle

In [2]:
API_KEY = "80113e47bfd1c58983acc18b4d46e025"  # this is a sample key
API_SECRET = "604648c3611f684b077c83df8e3a00fd"
username = "obi-1-konobe"
password_hash = pylast.md5("T4s4rt!rlastfm")
network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET,username=username, password_hash=password_hash)

In [3]:
artist = network.get_artist("Kurt Vile")
track = network.get_track(artist, 'Freeway')

In [4]:
artist_tags = artist.get_top_tags(10)
track_tags = track.get_top_tags(10)
similar_track = track.get_similar()

In [28]:
similar_track[0].item.artist.name

'Kurt Vile'

In [29]:
similar_track[0].item.title

'Breathin Out'

In [30]:
similar_track[0].match

1.0

In [37]:
track_tags[0].item.name

'rock'

In [38]:
track_tags[0].weight

'100'

In [42]:
track_tags[]

TopItem(item=pylast.Tag('american', pylast.LastFMNetwork('80113e47bfd1c58983acc18b4d46e025', '604648c3611f684b077c83df8e3a00fd', 'C7xrzJUwS7JsiDDxlSeJKIkskTYDeKBr', 'obi-1-konobe', '3ef6efe738937e23bbc45ed783ed10cd')), weight='34')

In [53]:
artist_tags[9].item.name

'folk rock'

In [54]:
artist_tags[9].weight

'11'

In [55]:
[(elem.item.name, elem.weight) for elem in artist_tags]

[('seen live', '100'),
 ('Lo-Fi', '89'),
 ('psychedelic', '77'),
 ('indie', '69'),
 ('bedroom pop', '31'),
 ('noise', '27'),
 ('singer-songwriter', '24'),
 ('folk', '22'),
 ('indie rock', '15'),
 ('folk rock', '11')]

In [30]:
with open('../data/pickles/4500.pickle', 'rb') as f:
    data = pickle.load(f)

In [31]:
len(data.keys())

4249

In [11]:
data[194]

{'artist_name': 'Ed Askew',
 'track_title': 'All I want to Do',
 'artist_tags': [('folk', '100'),
  ('psychedelic folk', '66'),
  ('psychedelic', '64'),
  ('singer-songwriter', '56'),
  ('experimental', '40'),
  ('acid folk', '16'),
  ('seen live', '14'),
  ('psych folk', '11'),
  ('american', '8'),
  ('people who are freakier and folkier than motherfucking devendra banhart',
   '8')],
 'track_tags': [],
 'similar_tracks': [('Ed Askew', 'safe harbor', 1.0),
  ('Ed Askew', 'A SHADOW OF CLOUDS', 1.0),
  ('Mark Fry', 'Walking In Paradise', 0.978),
  ('Mark Fry', "Ain't Got You", 0.978),
  ('The Incredible String Band', 'Vishangro', 0.977),
  ('Six Organs of Admittance', 'Close to the Sky', 0.974),
  ('Pearls Before Swine', 'Sail Away (Album Version)', 0.97),
  ('Pearls Before Swine', 'Mon Amour (Album Version)', 0.97),
  ('The Incredible String Band', 'Banjo Tune', 0.969),
  ('Mike Heron', 'Spirit Beautiful', 0.968),
  ('Simon Finn', 'Silent City Creep', 0.96),
  ('Simon Finn', "Loiterrin

In [24]:
asd = dict()
asd['{asd}'] = '{dsa}'
with open('asd.pickle', 'wb') as f:
    pickle.dump(asd, f, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
with open('asd.pickle', 'rb') as f:
    data = pickle.load(f, protocol=pickle.HIGHEST_PROTOCOL)

TypeError: 'protocol' is an invalid keyword argument for this function

In [26]:
data

{'{asd}': '{dsa}'}